In [2]:
from pyomo.environ import *
model = AbstractModel()

In [3]:
model.places = Param(within=NonNegativeIntegers)
model.transitions = Param(within=NonNegativeIntegers)

In [4]:
model.P = RangeSet(1, model.places)
model.T = RangeSet(1, model.transitions)

In [5]:
model.c = Param(model.P)
model.s = Param(model.T)
model.b = Param(model.T)
model.d = Param(model.P,model.T)
model.a = Param(model.P)

In [6]:
model.m1 = Var(model.P, domain=NonNegativeReals)
model.m2 = Var(model.P, domain=NonNegativeReals)
model.m3 = Var(model.P, domain=NonNegativeReals)

model.q1 = Var(model.T, domain=NonNegativeReals)
model.q2 = Var(model.T, domain=NonNegativeReals)

model.y1 = Var(model.T, domain=Binary)
model.y2 = Var(model.T, domain=Binary)

In [7]:
def obj_expression(model):
    return (summation(model.c, model.m3)+ summation(model.s,(model.q1+model.q2)) + summation(model.b, (model.y1+model.y2)))
model.OBJ = Objective(rule=obj_expression)

In [8]:
def bi_constraint_rule(model,t):
    return sum(model.y1[t])<=1 and sum(model.y2[t])<=1
model.binaryConstraint = Constraint(model.T, rule=bi_constraint_rule)

In [9]:
def sy_constraint_rule(model,t):
    return sum(model.y1[t]) <= sum(model.y2[t])
model.symmeryConstraint = Constraint(model.T, rule=sy_constraint_rule)

In [10]:
def bM_constraint_rule(model):
    return 0 <= sum(model.q1[t] for t in model.T) <= sum(500*model.y1[t] for t in model.T) and 0 <= sum(model.q2[t] for t in model.T) <= sum(500*model.y2[t] for t in model.T)
model.bMConstraint = Constraint(rule=bM_constraint_rule)

In [11]:
def bo_constraint_rule(model,p):
    return model.m1[p] == model.a[p]
model.boundConstraint = Constraint(model.P, rule=bo_constraint_rule)

In [12]:
def ma_constraint_rule(model,t):
    return (model.m2[p] for p in model.P)  == (model.m1[p]+sum(model.d[p,t]*model.q1[t] for p in model.P) for p in model.P) and (model.m3[p] for p in model.P)  == (model.m2[p]+sum(model.d[p,t]*model.q2[t] for p in model.P) for p in model.P)
model.markingConstraint = Constraint(model.T, rule=ma_constraint_rule)